# Fit Inspection

This notebook visually inspects light curve fits for each target in a given survey. 

#### Interesting fits from CSP

| Target | Model | Bands | notes |
|---|---|---|---|
| 2005M  | sn91bg | red | We would likely get a different fit result if we didn't crop the model. |
| 2005?  | sn91bg |  | Much better sn91bg fit with significantly different t0 than CSP (14 days) . |
| 2006dd  | sn91bg |  | Only caught the tail of the SN. |
| 2006et | sn91bg | red | chisq would be smaller with a larger stretch, but our model is limited. |
| 2006fw | | | One of the few CSP cases where the redshift is high enough to limit the number of "red" filters. |
| 2006gj | | red | A normal SN Ia (I think) with a better 91bg than Ia fit. |
| 2007jh, 2007ux, 2008R | salt2 | | The minimized t0 is visually too early, but increasing it results in a worse fit. |



In [ ]:
import sys
from pathlib import Path

import sndata
import yaml

sys.path.insert(0, '../')

from phot_class import fit_funcs
from phot_class import models
from phot_class import utils
from phot_class.classification import run_classification_fits


We begin by defining the data release to inspect light-curves for. We also define which parameters to vary while fitting and load a dictionary of priors from file.


In [ ]:
# Set this to the survey you want to inspect
# e.g. sndata.des.sn3yr, sndata.sdss.sako18
survey = sndata.csp.dr3

# Specify which model parameters you want to vary while fitting
# Options: 'z', t0', 'x0', 'x1', 'c'
vparams = ['t0', 'x0', 'x1', 'c']

# Specify the path where to load / save prior values from
config_path = Path('../priors_config.yml')


Next we build an iterable over the object ids for each supernova we want to fit.

In [ ]:
# Make sncosmo aware of the models and bandpasses we are using
models.register_sources(force=True)
survey.download_module_data()
survey.register_filters(force=True)

# Define an iterable over object ids
targets = survey.get_available_ids()
total_targets = len(targets)

target_iter = iter(targets)
band_names = survey.band_names
lambda_eff = survey.lambda_effective


Finally, we iterate over each supernova one at a time and fit each light-curve using the priors we loaded in from file earlier. To inspect fit results for a specific target, manually specify ``target_for_inspection`` as the desired object id.

In [ ]:
target_for_inspection = next(target_iter)
i = survey.get_available_ids().index(target_for_inspection)
print(f'{i + 1} / {total_targets}')


In [ ]:
print('Inspecting:', target_for_inspection)

# We reload the config data each time we fit. This means
# you can vary the parameters in the config file to
# improve the fit results in a semi-interactive way.
with open(config_path) as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)

# Load data. Note that we drop any data in the CSP NIR filters
# These filters tend to throw off our fits
target_data = survey.get_data_for_id(target_for_inspection, format_sncosmo=True)
target_data = target_data[target_data['band'] != 'csp_dr3_Ydw']
target_data = target_data[target_data['band'] != 'csp_dr3_Y']
z = target_data.meta['redshift']

# Format data for fitting
blue_data, red_data = utils.split_data(target_data, band_names, lambda_eff, z)
salt2_prior, salt2_kwargs, sn91bg_prior, sn91bg_kwargs = \
    utils.parse_config_dict(target_for_inspection, config)

band_sets = ['all', 'blue', 'red']
print('\nsalt2 prior:')
for b in band_sets:
    print(f'    {b}: {salt2_prior[b]}')
    
print('\nsalt2 kwargs:')
for b in band_sets:
    print(f'    {b}: {salt2_kwargs[b]}')
    
print('\nsn91bg prior:')
for b in band_sets:
    print(f'    {b}: {sn91bg_prior[b]}')
    
print('\nsn91bg kwargs:')
for b in band_sets:
    print(f'    {b}: {sn91bg_kwargs[b]}')

print()
fit_results = run_classification_fits(
    blue_data=blue_data,
    red_data=red_data,
    vparams=vparams,
    fit_func=fit_funcs.simple_fit,
    priors_s2=salt2_prior,
    priors_bg=sn91bg_prior,
    kwargs_s2=salt2_kwargs,
    kwargs_bg=sn91bg_kwargs,
    show_plots=True
)
